In [17]:
import numpy as np
from collections import defaultdict
import random

In [18]:
sig_matrix = np.arange(1000)
sig_matrix[[1,101, 201, 301, 401]] = sig_matrix[[0,100, 200, 300, 400]]
sig_matrix[[2,102, 202, 302, 402]] = sig_matrix[[4,104, 204, 304, 404]]
sig_matrix[[501,601, 701, 801, 901]] = sig_matrix[[505,605, 705, 805, 905]]
sig_matrix = sig_matrix.reshape(10, 100)
print(sig_matrix)
num_rows = sig_matrix.shape[0]
num_bands = 1
num_rows_band = num_rows / num_bands
sim_threshold = 0.8
possible_vals_sig_matrix = 1

[[  0   0   4   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
   18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
   36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
   54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
   72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
   90  91  92  93  94  95  96  97  98  99]
 [100 100 104 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
  118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
  136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
  154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
  172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
  190 191 192 193 194 195 196 197 198 199]
 [200 200 204 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
  218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
  236 237 238 239 240 241 

In [19]:
# Function number 6
def hash_function(vec, band_num):
    vec_str = ''.join(map(str, vec)) + str(band_num)
    return hash((vec_str, band_num))

def hash_band(band_matrix):
    hash_dict = defaultdict(list)
    for idx, col in enumerate(band_matrix.T):
        hash = hash_function(col, i)
        if hash.bit_length() > 32:
            hash = str(hash)
        hash_dict[hash].append(idx)
    return hash_dict

def lsh(sig_matrix, num_bands):
    num_rows = sig_matrix.shape[0]
    num_rows_band = num_rows / num_bands
    candidate_pairs = []
    for i in range(num_bands):
        band_matrix = sig_matrix[round(i * num_rows_band): round((i + 1) * num_rows_band)]
        hash_dict = hash_band(band_matrix)
        for _, value in hash_dict.items():
            if len(value) >= 2:
                #print(f"Hash: {key}, Indices: {value}")
                candidate_pairs.append(tuple(value))
    return candidate_pairs

In [20]:
#Functions from number 4
def generate_hash_functions(num_hash_funcs):
    p = 2**31 - 1
    hash_funcs = [(random.randint(0, 10**12), random.randint(0, 10**12), p) for _ in range(num_hash_funcs)]
    return hash_funcs

def approx_permutation(num_rows, hash_params):
    col = np.arange(num_rows)
    a, b, p = hash_params
    hash = ((a*col + b) % p) % num_rows
    return hash

def minHashing(col, hashed_col):
    min_hash = None
    for idx, hash_elem in enumerate(hashed_col):
        smaller_val_cond =  (min_hash == None or min_hash > hash_elem )
        if smaller_val_cond and col[idx] == 1:
            min_hash = hash_elem
    return min_hash


def minHash_signatures(input_data, hash_funcs):
    num_hash_funcs = len(hash_funcs)
    num_cols = input_data.shape[1]
    num_rows = input_data.shape[0]
    signature = np.empty((num_hash_funcs, num_cols))
    for idx_row, hash_params in enumerate(hash_funcs):
            hash_col = approx_permutation(num_rows, hash_params)
            for idx_col, col in enumerate(input_data.T):
                min_hash = minHashing(col, hash_col)
                signature[idx_row, idx_col] = min_hash
    return signature
# Function number 5
def generating_data(m,q,df, N):
    '''
    Input:
    - m: number of columns
    - q: number of non zero parameters
    - df: difference factor
    - N: rows

    Output:
    - ds: matrix as input for computation of signature matrix
    '''
    ds = np.zeros([N,m])
    random_int =[]
    for i in range(q):
        pos = random.randint(0,N-1)
        while pos in random_int:
            pos = random.randint(0,N-1)
        ds[pos,0]= 1
        random_int.append(pos)

    for column in range(1,m):
        current_positions = np.where(ds[:,column-1] == 1)[0]

        # Step 2: Determine the number of positions to replace
        replace_number =  int(df * q)

        # Step 3: Randomly select positions to replace
        replace_positions = np.random.choice(current_positions,  replace_number,replace=False)
        for i in replace_positions:
            ds[i,column] = 1
        # Step 4: Generate new unique positions
        available_positions = np.where(ds[:,column-1] == 0)[0]
        new_positions = np.random.choice(available_positions, q - replace_number,replace=False)
        for i in new_positions:
            ds[i,column] = 1

    return ds

def create_sig_matrix_routine(num_cols, num_rows, q, df, num_hash_funcs, ):
    hash_funcs = generate_hash_functions(num_hash_funcs)
    data = generating_data(num_cols, q, df, num_rows)
    sig_matrix = minHash_signatures(data, hash_funcs)
    return sig_matrix

In [21]:
#Test on a small scale
num_bands = 2
r = 5
num_cols = 20
num_rows = 10**2
q = 5
df = 0.1
num_hash_funcs = 6
test_sig_matrix = create_sig_matrix_routine(num_cols, num_rows, q, df, num_hash_funcs)
print(test_sig_matrix)

test = lsh(test_sig_matrix, 3)
print(test)

[[12.  5. 73. 10. 35. 25.  3. 34. 29. 13. 42. 15. 10. 17.  8. 22. 33.  5.
   1. 40.]
 [ 4. 47.  9. 26.  4. 40. 24. 12.  4. 26.  4.  7. 26. 15.  4.  1. 13. 44.
  12. 32.]
 [11. 37.  4. 21.  7.  7. 29. 37. 55.  7.  7. 25.  4.  7.  3.  0. 10.  7.
   3. 11.]
 [18. 48. 16. 17. 28. 23.  4. 14.  2.  7. 12. 55. 16.  4. 23.  0.  5.  2.
   9. 11.]
 [14.  2.  2. 12. 14. 43.  7.  7. 14. 50.  0.  0.  2.  7. 24.  2. 26.  0.
   2. 14.]
 [ 6.  3.  1. 24. 15. 25. 12. 11. 15.  1. 20. 27. 18. 12. 31. 10.  6.  6.
  13.  6.]]
[(3, 12), (2, 12), (0, 19), (4, 8), (6, 13)]


In [22]:
#c)
num_bands = 20
s = 0.8
num_cols = 100
num_rows = 10**4
q = 5
df = 0.1
num_hash_funcs = 20
test_sig_matrix = create_sig_matrix_routine(num_cols, num_rows, q, df, num_hash_funcs)
#print(test_sig_matrix)

candidates = lsh(test_sig_matrix, 3)
print(candidates)


[]
